# Реализация пятнашек :

In [ ]:
from math import sqrt
from heapq import heappop, heappush

# `a_star`
#
# `start_chain` - its a class with heuristic function in overload of `__lt__`
# and function `start.last_node()` which return a `node`
# `start_chain` has subclass 'node' with usefull info about graph edge (board state for 15 puzzle)
# and overload of `__eq__` which return result of last move in decision tree.
# `goal` - value which compare with `last_move`; its a target of this algh
# `last_node` - must be hashable uniq value
# `heapqpop` working as LIFO if keys are equal
# it means that algh will be worked as deep search in this case


def a_star(start_chain, goal_node):
    node_hash = {}
    chains_queue = []
    heappush(chains_queue, start_chain)
    while chains_queue:
        cur_chain = heappop(chains_queue)
        cur_node = cur_chain.last_node()
        if cur_node == goal_node:
            return cur_chain
        node_hash[cur_node] = cur_chain.g()
        for chain in cur_chain.get_neighbours():
            if chain.last_node() in node_hash:
                if chain.g() >= node_hash[chain.last_node()]:
                    continue
                node_hash[chain.last_node()] = chain.g()
            heappush(chains_queue, chain)

    raise Exception("No solution?!")


# Нахождение манхэттенского расстояния между а и b в матрице размера n
def manh_dst_matrix(a, b, n):
    return abs(a % n - b % n) + abs(a // n - b // n)

# Класс пятнашек
class chain15:
    # Функция вывода доски пятнашек
    def __str__(self):
        i = 0
        matrix = ""
        while i < self.size ** 2:
            matrix += str(self.board_state[i]) + " "
            if i % self.size == 3:
                matrix += "\n"
            i += 1
        return matrix

    # Инициализация

    def __init__(self, board_state, history=[]):
        self.board_state = list(board_state)
        self.size = int(sqrt(len(board_state)))
        self.history = history
        self.quad_size = int(self.size * self.size)

    def manh_dst(self):
        dst = 0
        for i in range(0, self.quad_size):
            dst += manh_dst_matrix((self.board_state[i] - 1) % self.quad_size, i, self.size)
        return int(dst)

    def last_node(self):
        # Должен являться хеширующемся значением. Список не хешируем.
        return str(self.board_state)

    # Один линейный конфликт увеличивает эвристику на 2. Здесь учитываются передвижения вверх и вниз.
    # Передвижения влево и вправо заложены в манхеттенское расстояние
    def linear_conflict(self):
        conflict_count = 0
        return 2 * conflict_count

    def last_move(self):
        if self.board_state[-1] == self.quad_size - 1 or self.board_state[-1] == self.quad_size - self.size:
            return 0
        return 2

    # Рассматриваем угловые ячейки (угловые конфликты?)
    def corner_tiles(self):
        conflict_count = 0
        # Левый верхний угол
        if self.board_state[0] != 1:
            if self.board_state[1] == 2 or self.board_state[self.size] == self.size + 1:
                conflict_count += 1
        # Правый верхний угол
        if self.board_state[self.size - 1] != self.size:
            if self.board_state[self.size - 2] == self.size - 1 or self.board_state[self.size * 2 - 1] == self.size * 2:
                conflict_count += 1
        # Левый нижний угол
        if self.board_state[self.quad_size - self.size] != self.quad_size - self.size + 1:
            if self.board_state[self.quad_size - self.size * 2] == self.quad_size - self.size * 2 + 1 or self.board_state[self.quad_size - self.size + 1] == self.quad_size - self.size + 2:
                conflict_count += 1
        # Правый нижний угол не рассматривается, т.к. он остается пустым
        return 2 * conflict_count

    # Эвристическое предположение о расстоянии от текущей вершины до терминальной
    def h(self):
        return self.manh_dst() + self.last_move()

    # Сумма расстояния от начальной вершины до текущей
    def g(self):
        return len(self.history)

    # Вес просматриваемой  вершины
    def f(self):
        return self.h() + self.g()

    def __lt__(self, other):
        return self.f() < other.f()

    def get_neighbours(self):
        neighs = []
        zero_coord = self.board_state.index(0)

        # Просмотр четырех возможных соседей 
        if zero_coord + 1 < self.size ** 2 and manh_dst_matrix(zero_coord, zero_coord + 1, self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord + 1] = new_state[zero_coord + 1], new_state[zero_coord]
            neighs.append(chain15(new_state, self.history + [self]))

        if zero_coord - 1 >= 0 and manh_dst_matrix(zero_coord, zero_coord - 1, self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord - 1] = new_state[zero_coord - 1], new_state[zero_coord]
            neighs.append(chain15(new_state, self.history + [self]))

        if zero_coord + self.size < self.size ** 2 and manh_dst_matrix(zero_coord, zero_coord + self.size,
                                                                       self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord + self.size] = new_state[zero_coord + self.size], new_state[
                zero_coord]
            neighs.append(chain15(new_state, self.history + [self]))

        if zero_coord - self.size >= 0 and manh_dst_matrix(zero_coord, zero_coord - self.size, self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord - self.size] = new_state[zero_coord - self.size], new_state[
                zero_coord]
            neighs.append(chain15(new_state, self.history + [self]))

        return neighs


if __name__ == '__main__':
    start_state = list(map(int, input("Введите начальное положение: ").split(", ")))
    generator_done = 0
    if generator_done == 0:
        zero_row = 0
        zr = start_state.index(0)
        if zr <= 3:
            zero_row = 1
        elif zr <= 7:
            zero_row = 2
        elif zr <= 11:
            zero_row = 3
        else:
            zero_row = 4
        print(zr + 1, zero_row)

        this_sum = 0

        for i in range(len(start_state) - 1):
            val1 = start_state[i]
            if val1 != 0:
                j = i + 1
                for j in range(len(start_state)):
                    val2 = start_state[j]
                    if val1 > val2 and val2 != 0: this_sum += 1

        this_sum += zero_row
        if start_state[14] == 15  and start_state[15] == 14 or this_sum % 2 == 0 :
            print("Данная комбинация нерешаема :(")
        else:
            print("Решение существует!")
            start = chain15(start_state)
            end = chain15((1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0))
            result = a_star(start, end.last_node())
            print(len(result.history))
            for node in result.history:
                print(node)
                print("-------------------------")
                
#В процессе работы алгоритма для вершин рассчитывается функция f(v)=g(v)+h(v)
#g(v)  — наименьшая стоимость пути в v из стартовой вершины,
#h(v) — эвристическое приближение стоимости пути от v до конечной цели.
#Эвристический алгоритм (эвристика) — алгоритм решения задачи, включающий практический метод,
#не являющийся гарантированно точным или оптимальным, но достаточный для решения поставленной задачи.
#Позволяет ускорить решение задачи в тех случаях, когда точное решение не может быть найдено.


Введите начальное положение: 3, 15, 12, 10, 9, 0, 11, 7, 8, 5, 4, 2, 14, 13, 6, 1
6 2
Решение существует!


Вывод: в ходе выполнения данной лабы, я научился понятиям "Эвристика", узнал про реализацию игры "Пятнашки", а также познакомился с некоторыми алгоритмами нахождения оптимального пути решения.